In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
df_segmentation = pd.read_csv('p002_1.csv', sep=';', decimal=",")
df_segmentation.head()

In [ ]:
df_segmentation['Abschluss'][0] == 5.0
df_segmentation.head()

In [ ]:
len(df_segmentation[(df_segmentation['Abschluss'] < 5)])

In [ ]:
for i in range (len(df_segmentation)):
    if df_segmentation['Abschluss'][i] < 5.0:
        df_segmentation['Abschluss'][i] = 1
    else:
        df_segmentation['Abschluss'][i] = 0
df_segmentation.head()

In [ ]:
df_segmentation

In [ ]:
scaler = StandardScaler()
num_columns = 15

segmentation_std = scaler.fit_transform(df_segmentation)
abschluss = df_segmentation['Abschluss']
Bundesland = df_segmentation['Bundesland']
JahreseinkommenderEltern = df_segmentation['Jahreseinkommen der Eltern']
Staatsbürgerschaft = df_segmentation['Staatsbürgerschaft']
Schnitt = df_segmentation['Schnitt']
Geschlecht = df_segmentation['Geschlecht']
del df_segmentation['Abschluss']
del df_segmentation['Bundesland']
del df_segmentation['Jahreseinkommen der Eltern']
del df_segmentation['Staatsbürgerschaft']
del df_segmentation['Schnitt']
del df_segmentation['Geschlecht']
segmentation_std = scaler.fit_transform(df_segmentation)
df_segmentation

In [ ]:
segmentation_std

In [ ]:
pca = PCA()
pca.fit(segmentation_std)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.figure(figsize = (10, 8))
plt.plot(range(0,10), pca.explained_variance_ratio_.cumsum(), marker = 'o', linestyle = '--')
plt.title('Explained Variance by Components')
plt.xlabel('number of Components')
plt.ylabel('Cumulative Explained Variance')

In [ ]:
pca = PCA(n_components = 3)

In [ ]:
pca.fit(segmentation_std)

In [ ]:
pca.transform(segmentation_std)

help(PCA)

In [ ]:
pca.components_

### Hier ist die Component 1 (erste Array) wichtig zum Entscheiden

In [ ]:
head = df_segmentation.columns.values

In [ ]:
summe = sum(abs(pca.components_[0]))
z = 0
for i in pca.components_[0]:
    a = round(abs(i)/summe, 3)
    print(head[z] + ': ', a)
    z += 1

In [ ]:
scores_pca = pca.transform(segmentation_std)

In [ ]:
(sum(pca.components_[0]))/10

In [ ]:
df_segmentation

In [ ]:
wcss = []
for i in range(1, 10):
    kmeans_pca = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans_pca.fit(scores_pca)
    wcss.append(kmeans_pca.inertia_)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(range(1, 10), wcss, marker='o', linestyle='--')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.title('K-means with PCA Clustering')
plt.show()

In [ ]:
kmeans_pca = KMeans(n_clusters=4, init='k-means++', random_state=42)

In [ ]:
kmeans_pca.fit(scores_pca)

In [ ]:
df_segm_pca_kmeans = pd.concat([df_segmentation.reset_index(drop = True), pd.DataFrame(scores_pca)], axis = 1)
df_segm_pca_kmeans.columns.values[-3:] = ['Component 1', 'Component 2', 'Component 3']
len(kmeans_pca.labels_)
df_segm_pca_kmeans['Segment K-means PCA'] = kmeans_pca.labels_

In [ ]:
df_segm_pca_kmeans

In [ ]:
df_segment_pca_kmeans = pd.concat([df_segm_pca_kmeans, abschluss], axis = 1)

In [ ]:
df_segm_pca_kmeans['Segment'] = df_segm_pca_kmeans['Segment K-means PCA'].map({0:'first',
                                                                               1:'second',
                                                                               2:'third',
                                                                               3:'fourth'})

In [ ]:
df_segmentation

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
x_axis = df_segm_pca_kmeans['Component 2']
y_axis = df_segm_pca_kmeans['Component 3']
z_axis = df_segm_pca_kmeans['Component 1']
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())
ax.scatter(x_axis, y_axis, z_axis, cmap=['r', 'g','b'])
plt.title('Cluster by PCA Components')
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
x_axis = df_segm_pca_kmeans['Component 2']
y_axis = df_segm_pca_kmeans['Component 1']
plt.figure(figsize = (11, 8))
cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())
sns.scatterplot(x_axis, y_axis, hue = df_segm_pca_kmeans['Segment'], cmap="brg")
plt.title('Cluster by PCA Components')
plt.show()

### Hier sieht man dass, die Component 1 am meisten Aussage hat. D.h. wenn man Component 1 weg lässt, dann kann man keine Kategorie erzeugen.